Import necessary packages

In [2]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression

In [3]:
train_set = pd.read_csv('/home/gigi/grive/InsightDataScience/Click_through_Rate/train.csv', sep = ',', nrows=99999)
print np.mean(train_set['click']) # is 0.1749
CV_set = pd.read_csv('/home/gigi/grive/InsightDataScience/Click_through_Rate/train.csv', sep = ',', skiprows = 99999, nrows = 99999, )
test_set = pd.read_csv('/home/gigi/grive/InsightDataScience/Click_through_Rate/test.csv', sep = ',', nrows=99999)
CV_set.columns = train_set.columns
train_set.head()

/home/gigi/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


0.174901749017


,id,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,...,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
0,1000009418151094273,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,2,15706,320,50,1722,0,35,-1,79
1,10000169349117863715,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,0,15704,320,50,1722,0,35,100084,79
2,10000371904215119486,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,0,15704,320,50,1722,0,35,100084,79
3,10000640724480838376,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,0,15706,320,50,1722,0,35,100084,79
4,10000679056417042096,0,14102100,1005,1,fe8cc448,9166c161,0569f928,ecad2386,7801e8d9,...,1,0,18993,320,50,2161,0,35,-1,157


In [4]:
train_dev = train_set.groupby('device_type').mean()['click']
train_C1 = train_set.groupby('C1').mean()['click']

unique, counts = np.unique(train_set['device_conn_type'], return_counts=True) 
train_conn_freq = pd.DataFrame({'device_conn_type': unique,'Frequencies': counts})

unique, counts = np.unique(train_set['C15'], return_counts=True)
train_C15_freq = pd.DataFrame({'C15': unique,'Frequencies': counts})

unique, counts = np.unique(train_set['C16'], return_counts=True) 
train_C16_freq = pd.DataFrame({'C16': unique,'Frequencies': counts})

Know the datatype of each column.

In [5]:
data_type = pd.DataFrame({'Column':train_set.columns, 'type':[type(i) for i in train_set.columns]})
print data_type

              Column          type
0                 id  <type 'str'>
1              click  <type 'str'>
2               hour  <type 'str'>
3                 C1  <type 'str'>
4         banner_pos  <type 'str'>
5            site_id  <type 'str'>
6        site_domain  <type 'str'>
7      site_category  <type 'str'>
8             app_id  <type 'str'>
9         app_domain  <type 'str'>
10      app_category  <type 'str'>
11         device_id  <type 'str'>
12         device_ip  <type 'str'>
13      device_model  <type 'str'>
14       device_type  <type 'str'>
15  device_conn_type  <type 'str'>
16               C14  <type 'str'>
17               C15  <type 'str'>
18               C16  <type 'str'>
19               C17  <type 'str'>
20               C18  <type 'str'>
21               C19  <type 'str'>
22               C20  <type 'str'>
23               C21  <type 'str'>


Set the train and cross validation and test data with four features.

In [12]:
X = train_set[['C1','device_type', 'C15', 'C16']]
y = train_set['click'].astype(float)
X_CV = CV_set[['C1','device_type', 'C15', 'C16']]
y_CV = CV_set['click'].astype(float)
X_test = test_set[['C1','device_type', 'C15', 'C16']]
X.head()

,C1,device_type,C15,C16
0,1005,1,320,50
1,1005,1,320,50
2,1005,1,320,50
3,1005,1,320,50
4,1005,1,320,50


In [7]:
LogReg = LogisticRegression()
LogReg.fit(X, y)
Score_Log_l2 = LogReg.score(X_CV,y_CV)

LogReg = LogisticRegression(penalty='l1') 
LogReg.fit(X, y)
Score_Log_l1 = LogReg.score(X_CV,y_CV) 
print Score_Log_l2, Score_Log_l1

0.825288252883 0.825288252883


In [8]:
from sklearn.ensemble import RandomForestClassifier
RD = RandomForestClassifier(n_estimators=10) 
RD.fit(X, y) 
Score_RD = RD.score(X_CV,y_CV)
y_predict_RD = RD.predict(X_test[:10])
print Score_RD

0.825388253883


In [9]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X,y)
y_predict_prob = gnb.predict_proba(X_test[:10])

In [10]:
y_predict_log = LogReg.predict(X_test[:10])
y_predict_prob = LogReg.predict_proba(X_test[:10])

In [15]:
# feature selection
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LassoCV
clf = LassoCV()
sfm = SelectFromModel(clf, threshold = 0.25)
# drop all non relevant non number features
X = train_set.drop(['site_id','site_domain','site_category','app_id','app_domain','app_category','device_id','device_ip','device_model','device_type','device_conn_type'],axis = 1)
for i in range(len(X.columns)):
    X[X.columns[i]] = X[X.columns[i]].astype(float)
X.head()

,id,click,hour,C1,banner_pos,C14,C15,C16,C17,C18,C19,C20,C21
0,1.000009e+18,0.0,14102100.0,1005.0,0.0,15706.0,320.0,50.0,1722.0,0.0,35.0,-1.0,79.0
1,1.000017e+19,0.0,14102100.0,1005.0,0.0,15704.0,320.0,50.0,1722.0,0.0,35.0,100084.0,79.0
2,1.000037e+19,0.0,14102100.0,1005.0,0.0,15704.0,320.0,50.0,1722.0,0.0,35.0,100084.0,79.0
3,1.000064e+19,0.0,14102100.0,1005.0,0.0,15706.0,320.0,50.0,1722.0,0.0,35.0,100084.0,79.0
4,1.000068e+19,0.0,14102100.0,1005.0,1.0,18993.0,320.0,50.0,2161.0,0.0,35.0,-1.0,157.0


In [18]:
import seaborn as sns 
import matplotlib.pyplot as plt 
sns.set(context="paper", font="monospace")  
corrmat = X.corr()  
# Set up the matplotlib figure 
f, ax = plt.subplots(figsize=(12, 9))  
# Draw the heatmap using seaborn 
sns.heatmap(corrmat, vmax=.8, square=True)  
f.tight_layout()
plt.show()

Rerun the model using the selected 7 features according to their correlation and weight contribution.

In [19]:
X = train_set[['C1','device_conn_type', 'C15', 'C16','C17','C19','C20']] 
X_CV = CV_set[['C1','device_conn_type', 'C15', 'C16','C17','C19','C20']] 
X_test = test_set[['C1','device_conn_type', 'C15', 'C16','C17','C19','C20']]  
LogReg = LogisticRegression() 
LogReg.fit(X, y) 
Score_Log_l2 = LogReg.score(X_CV,y_CV)  
LogReg = LogisticRegression(penalty='l1')  
LogReg.fit(X, y) 
Score_Log_l1 = LogReg.score(X_CV,y_CV)
print Score_Log_l2, Score_Log_l1

0.824048240482 0.823728237282
